In [1]:
%cd /spark-data/capstone_crm/etl

%run 'load_data.ipynb'
%run 'verify_data.ipynb'
%run 'data_cleaning.ipynb'

%cd /spark-data/capstone_crm/utils
%run 'logger_setup.ipynb'



/.local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/spark-data/capstone_crm/etl
/spark-data/capstone_crm/utils


/.local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
log_file = os.path.join("../logs", "pipeline.log")
setup_logging(log_file)

In [3]:
data_files = ["customers", "interactions", "products", "sales_team", "transactions", "country_codes"]
data_dictionary = load_data(data_files)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/07 14:16:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-09-07 14:16:23,517 - INFO : customers data loaded successfully.
2024-09-07 14:16:23,580 - INFO : interactions data loaded successfully.
2024-09-07 14:16:23,629 - INFO : products data loaded successfully.
2024-09-07 14:16:23,679 - INFO : sales_team data loaded successfully.
2024-09-07 14:16:23,736 - INFO : transactions data loaded successfully.
2024-09-07 14:16:23,780 - INFO : country_codes data loaded successfully.


In [4]:
data_dictionary

{'customers': DataFrame[Customer_ID: string, Name: string, Email: string, Phone: string, Country: string],
 'interactions': DataFrame[Interaction_ID: string, Customer_ID: string, Interaction_Date: date, Interaction_Type: string, Issue_Resolved: boolean],
 'products': DataFrame[Product_ID: int, Product_Name: string, Category: string, Price: double],
 'sales_team': DataFrame[Sales_Rep_ID: string, Name: string, Region: string, Sales_Target: double, Sales_Achieved: double],
 'transactions': DataFrame[Transaction_ID: string, Customer_ID: string, Product_ID: int, Date: date, Amount: double, Sales_Rep_ID: string],
 'country_codes': DataFrame[Country: string, Country_Code: string]}

In [5]:
data_dictionary["customers"] = join_customer_country_code(data_dictionary["customers"],data_dictionary["country_codes"]) 

In [6]:
#show_data(data_dictionary)
print_schema(data_dictionary)

2024-09-07 14:16:24,061 - INFO : Printing schemas for all dataframes


root
 |-- Customer_ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country_Code: string (nullable = true)

root
 |-- Interaction_ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Interaction_Date: date (nullable = true)
 |-- Interaction_Type: string (nullable = true)
 |-- Issue_Resolved: boolean (nullable = true)

root
 |-- Product_ID: integer (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Price: double (nullable = true)

root
 |-- Sales_Rep_ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Sales_Target: double (nullable = true)
 |-- Sales_Achieved: double (nullable = true)

root
 |-- Transaction_ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Product_ID: integer (nullable = true)
 

In [7]:
# show_data(data_dictionary)

In [8]:
if verify_data_accuracy(data_dictionary):
    print("Data accuracy Verified.")
else:
    print("Data accuracy is not Verified.")

2024-09-07 14:16:24,111 - INFO : Starting data accuracy verification
2024-09-07 14:16:26,733 - INFO : Invalid customer records in transactions: 0
2024-09-07 14:16:27,329 - INFO : Invalid customer records in interactions: 0
2024-09-07 14:16:27,740 - INFO : Invalid product records in transactions: 0
2024-09-07 14:16:27,999 - INFO : Invalid sales rep records in transactions: 0
2024-09-07 14:16:28,000 - INFO : Data accuracy verified: All records are valid


Data accuracy Verified.


In [9]:

data_dictionary = clean_data(data_dictionary)

2024-09-07 14:16:28,014 - INFO : Cleaning customers dataframe
2024-09-07 14:16:28,017 - INFO : Checking for missing values
2024-09-07 14:16:28,106 - INFO : Handling missing values
2024-09-07 14:16:28,201 - INFO : Removing duplicates
2024-09-07 14:16:28,433 - INFO : 525 total records.
2024-09-07 14:16:29,216 - INFO : 507 records left after dropping duplicates.
2024-09-07 14:16:29,218 - INFO : Correcting inaccurate data
2024-09-07 14:16:29,323 - INFO : Successfully cleaned data for customers dataFrame
2024-09-07 14:16:29,325 - INFO : .....................................................................................................................
2024-09-07 14:16:29,327 - INFO : Cleaning interactions dataframe
2024-09-07 14:16:29,328 - INFO : Checking for missing values
2024-09-07 14:16:29,388 - INFO : Handling missing values
2024-09-07 14:16:29,448 - INFO : Removing duplicates
2024-09-07 14:16:29,603 - INFO : 832 total records.
2024-09-07 14:16:30,053 - INFO : 806 records left after 

In [10]:
show_data(data_dictionary)

+--------------------+--------------------+--------------------+-----------+--------------------+------------+
|         Customer_ID|                Name|               Email|      Phone|             Country|Country_Code|
+--------------------+--------------------+--------------------+-----------+--------------------+------------+
|489f42c9-55f0-44d...|           Jeff King|chaneyricky@examp...| 2976896759|              Tuvalu|        +688|
|dc1d043c-1085-4f9...|Christopher Anderson|brittany18@exampl...|19385699876|Saint Pierre and ...|          NA|
|9d32e2e9-9d94-422...|       Kevin Alvarez|isaiah98@example.org| 9296728348|              Serbia|        +381|
|5130890c-8225-412...|   Crystal Rodriguez|                  NA| 9007005604|             Iceland|        +354|
|bab75d30-5312-40a...|        Lucas Jordan|jamescummings@exa...|         NA| Antigua and Barbuda|          NA|
+--------------------+--------------------+--------------------+-----------+--------------------+------------+
o

24/09/08 06:07:52 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 688610 ms exceeds timeout 120000 ms
24/09/08 06:07:52 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/08 06:07:56 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$